# Bond Pricing

This notebook demonstrates how to price Bonds using the Instrument Pricing Analytics (IPA) and the Refinitiv Data Libraries.

##### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [logging](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) into the Refinitiv Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/tutorials), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/documentation)
 and much more.

You may also visit the [API Playground](https://apidocs.refinitiv.com/Apps/ApiDocs) to get more examples, find  end points and download Swagger files

##### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be happy to help. 

## Import the library and connect to Refinitiv Data Libraries

In [1]:
import refinitiv.data as rd
import refinitiv.data.content.ipa.financial_contracts as rdf
from refinitiv.data.content.ipa.financial_contracts import bond

## Open the data session

The open_session() function creates and open sessions based on the information contained in the refinitiv-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [2]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x105c902b0 {name='workspace'}>

## Bond Pricing

The get_bond_analytics function computes bond analytics (yield, sensitivities, spreads) based on the latest available market data or using end of day data.

The list of available fields can be found on the Refinitiv Developer Community [here](https://developers.refinitiv.com/refinitiv-data-platform/refinitiv-data-platform-apis/docs?content=72362&type=documentation_item)


In [3]:
response = bond.Definition(

    instrument_code="US5YT=RR",

    fields=[
        "InstrumentDescription",
        "MarketDataDate",
        "Price",
        "YieldPercent",
        "GovernmentSpreadBp",
        "SectorRatingSpreadBp"
    ]
).get_data()

response.data.df


,InstrumentDescription,MarketDataDate,Price,YieldPercent,GovernmentSpreadBp,SectorRatingSpreadBp
0,UST 4.000 02/29/28,2023-03-07 06:16:11,98.910156,4.244977,0,<NA>


### Single Bond

In [4]:
response = bond.Definition(

    instrument_code="13063CUV0",
    fields=[
        "InstrumentDescription",
        "MarketDataDate",
        "Price",
        "YieldPercent",
        "GovernmentSpreadBp",
        "SectorRatingSpreadBp"
    ]
).get_data()

response.data.df


,InstrumentDescription,MarketDataDate,Price,YieldPercent,GovernmentSpreadBp,SectorRatingSpreadBp
0,CA 5.000 08/01/28 '25,2023-03-07 06:16:14,105.289,2.703,-205,-44.7


### Multiple Bonds

In [5]:
response = rdf.Definitions(

    universe=[
        bond.Definition(instrument_code="US1YT=RR"),
        bond.Definition(instrument_code="US5YT=RR"),
        bond.Definition(instrument_code="US10YT=RR")],

    fields=["InstrumentDescription",
            "MarketDataDate",
            "Price",
            "YieldPercent",
            "ZSpreadBp"]
).get_data()

response.data.df

,InstrumentDescription,MarketDataDate,Price,YieldPercent,ZSpreadBp
0,UST Bill 02/22/24,2023-03-07 06:16:18,95.316344,5.048614,-54.256602
1,UST 4.000 02/29/28,2023-03-07 06:16:18,98.910156,4.244977,-7.1626
2,UST 3.500 02/15/33,2023-03-07 06:16:18,96.304688,3.952895,-0.207418


### Global pricing parameters

The pricing parameters can be used to control the calculation. The list of available parameters as well as available fields can be found on the Refinitiv Developers Community [here](https://developers.refinitiv.com/refinitiv-data-platform/refinitiv-data-platform-apis/docs?content=72362&type=documentation_item)


In [6]:
response = rdf.Definitions(

    universe=[
        bond.Definition(instrument_code="US1YT=RR"),
        bond.Definition(instrument_code="US5YT=RR"),
        bond.Definition(instrument_code="US10YT=RR")
    ],
    pricing_parameters=bond.PricingParameters(
        market_data_date="2020-07-05",
        price_side=bond.PriceSide.BID
    ),
    fields=[
        "InstrumentDescription",
        "MarketDataDate",
        "Price",
        "YieldPercent",
        "ZSpreadBp"]
).get_data()

response.data.df

,InstrumentDescription,MarketDataDate,Price,YieldPercent,ZSpreadBp
0,UST Bill 06/17/21 MATd,2020-07-05,99.851028,0.157329,-14.379905
1,UST 0.250 06/30/25,2020-07-05,99.765625,0.297412,-3.367682
2,UST 0.625 05/15/30,2020-07-05,99.5625,0.670924,2.91446


### Global & individual pricing parameters
The pricing parameters may also be defined at the instrument level and combined with global prameters. In the example below the 3 bonds are priced at the same date in the past but each bond is assigned a specifc price.

In [7]:
response = rdf.Definitions(

    universe=[
        bond.Definition(
            instrument_code="FR5YT=RR",
            pricing_parameters=bond.PricingParameters(price=102)
        ),
        bond.Definition(
            instrument_code="FR10YT=RR",
            pricing_parameters=bond.PricingParameters(price=100)),
        bond.Definition(
            instrument_code="FR20YT=RR",
            pricing_parameters=bond.PricingParameters(price=180)
        )
    ],
    pricing_parameters=bond.PricingParameters(
        market_data_date="2020-07-05",
        price_side=bond.PriceSide.BID
    ),

    fields=[
        "InstrumentDescription",
        "MarketDataDate",
        "Price",
        "YieldPercent",
        "ZSpreadBp"]
).get_data()

response.data.df


,InstrumentDescription,MarketDataDate,Price,YieldPercent,ZSpreadBp
0,FRGV 0.500 05/25/25,2020-07-05,102,0.089267,44.046415
1,FRGV 11/25/30,2020-07-05,100,0.0,14.241639
2,FRGV 0.500 05/25/40,2020-07-05,180,-2.541922,-263.452759


### User defined Bond 

In addition it is also possible to model a bond by defining it parmaters. the list of available parameters can be found [here](https://developers.refinitiv.com/refinitiv-data-platform/refinitiv-data-platform-apis/docs?content=72362&type=documentation_item)

In [8]:
response = rdf.Definitions(

    universe=[
        bond.Definition(
            issue_date="2020-02-28",
            end_date="2024-02-28",
            notional_ccy="EUR",
            notional_amount=10000,
            interest_payment_frequency="Zero",
            fixed_rate_percent=0,
            interest_calculation_method="Dcb_Actual_Actual",
            pricing_parameters=bond.PricingParameters(yield_percent=0.1)),
    ],

    fields=[
        "InstrumentDescription",
        "MarketDataDate",
        "MarketValueInDealCcy",
        "Price",
        "YieldPercent",
        "ModifiedDuration"]
).get_data()

response.data.df


,InstrumentDescription,MarketDataDate,MarketValueInDealCcy,Price,YieldPercent,ModifiedDuration
0,<NA>,2023-03-07 06:16:28,9990.201485,99.902015,0.1,0.979842


## Close the default session

In [9]:
rd.close_session()